In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import json
from decouple import config
from urllib import request
import csv 
import datetime
import smtplib
from pandas import json_normalize
from sqlalchemy import create_engine
import psycopg2
from email.message import EmailMessage


# f = open('/Users/dreamscape08/Desktop/Osteostrong Project/OSPGRM_files/lw/lw-users.json')
# lw_data = json.load(f)
def dateme():
    parser = datetime.datetime.now()
    return parser.strftime("%m-%d-%Y")

lw_username = config('lw_username')
lw_password = config('lw_password')

lw_payload = {'username': lw_username, 'password': lw_password}
loginurl = ('https://osteostrong.org/auth')
lw_totals = []

with requests.session() as s:
    s.post(loginurl, data=lw_payload)
    lw_url = f"https://osteostrong.org/user/users?LocationId=00053"
    lw_r = s.get(lw_url).text
    jsondata=json.loads(lw_r)    
    df=pd.json_normalize(jsondata)
    alluserslw=df[['FirstName.S', 'LastName.S', 'Email.S', 'Phone.S','UserId.S','NfcId.S']]#.to_csv('/Users/dreamscape08/Desktop/Osteostrong Project/OSPGRM_files/lw/lw-users.csv')
    for x in alluserslw.index:
        usrid=alluserslw.loc[x,'UserId.S']
        lw_url = f"https://osteostrong.org/chart?UserId={usrid}&LocationId=00053"
        lw_r = s.get(lw_url)
        lw_soup = BeautifulSoup(lw_r.content, 'html.parser')
        lw_total_sesh = lw_soup.find_all('div', attrs={'class': 'col-1'})
        lw_Total_Sessions = (lw_total_sesh[2].text.strip())
        lw_sesh_totals = int(lw_Total_Sessions)
        lw_totals.append([lw_sesh_totals, usrid])
s.close()
new=pd.DataFrame(lw_totals,columns=['Total Sessions','UserId']).set_index('UserId')
alluserslw.set_index('UserId.S', inplace=True)
lwULTRA=alluserslw.join(new)
lwULTRA.reset_index(inplace=True)

lwULTRA.columns=['UserID','FirstName','LastName','Email','Phone','NfcId','Total Sessions']

lwULTRA.to_excel(f'/Users/dreamscape08/Desktop/Osteostrong Project/OS_Operations/lw_Clientdb_{dateme()}.xlsx', header=True,index=False)
alert=[]
list = [4,9,24,49,74,99,124,149,199]
for k, v in lwULTRA['Total Sessions'].items():
    if v in list:
        alert.append(lwULTRA.loc[k,['FirstName','LastName','Total Sessions']])
alert=pd.DataFrame(alert,columns=['FirstName','LastName','Total Sessions']).sort_values(by='Total Sessions',ascending=False)
alert.reset_index(drop=True,inplace=True)
alert.to_excel(f'/Users/dreamscape08/Desktop/Osteostrong Project/OS_Operations/lw_alert_{dateme()}.xlsx', header=True,index=False)

username = config('pg_username')
password = config('pg_password')
host = 'localhost'
port = '5432'
database = 'osteostrong'
postgres_str = f'postgresql://{username}:{password}@{host}:{port}/{database}'
cnx=create_engine(postgres_str)
lwULTRA.to_sql('lw_Clientdb', cnx, if_exists='replace',index=False)
operations_manager = config('opm_email')
sender_proton = config('proton_email')
p_key = config('proton_key')
to = f'{operations_manager}'
admin_email = f'{sender_proton}' # (You should provide your gmail account name)
admin_pass = f'{p_key}' # (You should provide your gmail password)
# smtpserver = smtplib.SMTP('localhost',1025)
# smtpserver.ehlo()
# smtpserver.starttls()
# smtpserver.ehlo
# smtpserver.login(admin_email, admin_pass)
# header = 'To:' + to + '\n' + 'From: ' + admin_email + '\n' + f'Subject:lw BASELINE ALERTS-{dateme()} \n'
# msg = header + f'\n {alert} \n\n'
# print(header)
# smtpserver.sendmail(admin_email, to, msg)
# print('done!')
# smtpserver.close() 
pathnow=f'/Users/dreamscape08/Desktop/Osteostrong Project/OS_Operations/lw_alert_{dateme()}.xlsx'
def send_mail_with_excel():
    msg = EmailMessage()
    msg['Subject'] = f'LW FUB ALERTS-{dateme()}'
    msg['From'] = admin_email
    msg['To'] = to
    msg.set_content(f'{alert}')

    with open(pathnow, 'rb') as f:
        file_data = f.read()
    msg.add_attachment(file_data, maintype="application", subtype="vnd.openxmlformats-officedocument.spreadsheetml.sheet", filename=f'Lakewood_FUB_{dateme()}.xlsx')

    with smtplib.SMTP('localhost',1025) as smtpserver:
        smtpserver.ehlo()
        smtpserver.starttls()
        smtpserver.ehlo
        smtpserver.login(admin_email, admin_pass)
        smtpserver.send_message(msg)
        print('Email sent!')
        smtpserver.close() 
send_mail_with_excel()



Email sent!
